In [46]:
!pip install requests
!pip install pandas

In [17]:
import requests
import pandas as pd

In [94]:
# PARAMETROS DE ENTRADA
WORK_MODE = []
SECTOR = []
EDUCATION = []
SALARY = []
HOURS = [3]
CONTRACT_TYPE = []
RECENCY = []

BASE_API_URL = "https://mtpe-candidatos.empleosperu.gob.pe/api/jobs"

LIMIT = 0
OFFSET = 0

In [95]:
filters = []

for mode in WORK_MODE:
    filters.append(f"WORK_MODE=={mode}")

for sector in SECTOR:
    filters.append(f"SECTOR=={sector}")

for education in EDUCATION:
    filters.append(f"EDUCATION=={education}")

for salary in SALARY:
    filters.append(f"SALARY=={salary}")

for hour in HOURS:
    filters.append(f"HOURS=={hour}")

for contract in CONTRACT_TYPE:
    filters.append(f"CONTRACT_TYPE=={contract}")

for recency in RECENCY:
    filters.append(f"RECENCY=={recency}")

filters_str = ','.join(filters) if filters else ''

if filters_str:
    API_FILTER = f"{BASE_API_URL}?facets={filters_str}&limit={LIMIT}&offset={OFFSET}"
else:
    API_FILTER = f"{BASE_API_URL}?limit={LIMIT}&offset={OFFSET}"

print(API_FILTER)

https://mtpe-candidatos.empleosperu.gob.pe/api/jobs?facets=HOURS==3&limit=0&offset=0


In [96]:
raw_data = requests.get(API_URL)

In [9]:
data = raw_data.json()

In [34]:
vacancies_list = []
facets_list = []

In [35]:
for vacancy in data['data']:
    vacancy_data = {
        "IdVacante": vacancy['id'],
        "TituloPosicion": vacancy['positionTitle'],
        "Pais": vacancy['location']['COUNTRY'],
        "Departamento": vacancy['location']['DEPARTMENT'],
        "Provincia": vacancy['location']['PROVINCE'],
        "Distrito": vacancy['location']['DISTRICT'],
        "Longitud": vacancy['geoLocation']['longitude'],
        "Latitud": vacancy['geoLocation']['latitude'],
        "IdCompania": vacancy['companyId'],
        "NombreCompania": vacancy['companyName'],
        "DescripcionCompania": vacancy['companyDescription'],
        "TipoContrato": vacancy['contractType']['name'],
        "HorasPorSemana": vacancy['hoursPerWeek']['name'],
        "MaxHoras": vacancy['hoursPerWeek'].get('maximum', 'No especificado'),
        "GradoEducacion": vacancy['educationDegree']['name'],
        "MinSalario": vacancy['offeredRemunerationPackages'][0].get('minimumAmount', 'No especificado') if vacancy['offeredRemunerationPackages'] else 'No especificado',
        "MaxSalario": vacancy['offeredRemunerationPackages'][0].get('maximumAmount', 'No especificado') if vacancy['offeredRemunerationPackages'] else 'No especificado',
        "TipoSalario": vacancy['offeredRemunerationPackages'][0]['type'] if vacancy['offeredRemunerationPackages'] else 'No especificado',
        "MostrarSalario": vacancy['displaySalary'],
        "FechaPublicacion": vacancy['datePosted'],
        "EstadoAprovacion": vacancy['approvalStatus']
    }
    vacancies_list.append(vacancy_data)

In [ ]:
for general in data['facets']:
    facet_id = general['id']
    facet_name = general['name']
    multi_selectable = general['multiSelectable']
    searchable = general['searchable']
    
    for option in general['options']:
        option_data = {
            "facet_id": facet_id,
            "facet_name": facet_name,
            "multi_selectable": multi_selectable,
            "searchable": searchable,
            "option_id": option['id'],
            "option_name": option['name'],
            "number_of_matches": option['numberOfMatches'],
            "filter_data": option['filterData']
        }
        
        facets_list.append(option_data)


In [37]:
facets_df = pd.DataFrame(facets_list)
vacancies_df = pd.DataFrame(vacancies_list)

In [56]:
vacancies_df['Provincia']

0    1505
1    1505
2    1603
Name: Provincia, dtype: object

In [ ]:
facets_dfs = {}

for general in data['facets']:
    facet_id = general['id']
    facet_name = general['name']
    
    options_list = []
    
    for option in general['options']:
        option_data = {
            "option_id": option['id'],
            "option_name": option['name'],
            "number_of_matches": option['numberOfMatches'],
            "filter_data": option['filterData']
        }
        
        options_list.append(option_data)
    
    facets_dfs[facet_id] = pd.DataFrame(options_list)

In [45]:
facets_dfs['SALARY']

,option_id,option_name,number_of_matches,filter_data
0,A,Hasta 1024,37,SALARY==A
1,B,1025 - 1199,277,SALARY==B
2,C,1200-1399,137,SALARY==C
3,D,1400-1599,96,SALARY==D
4,E,1600-1799,57,SALARY==E
5,F,1800-1999,45,SALARY==F
6,G,2000-2499,47,SALARY==G
7,H,2500-2999,30,SALARY==H
8,I,3000-3499,26,SALARY==I
9,J,3500-3999,14,SALARY==J


In [47]:
facets_dfs['WORK_MODE']

,option_id,option_name,number_of_matches,filter_data
0,1,Remoto,11,WORK_MODE==1
1,2,Presencial,1104,WORK_MODE==2
2,3,Mixto,38,WORK_MODE==3


In [48]:
facets_dfs['EDUCATION']

,option_id,option_name,number_of_matches,filter_data
0,1,Sin estudios,16,EDUCATION==1
1,10,Universitaria completa,107,EDUCATION==10
2,11,Universitaria - Bachiller,64,EDUCATION==11
3,12,Universitaria - Licenciado,73,EDUCATION==12
4,13,Universitaria incompleta,0,EDUCATION==13
5,14,Maestría completa,5,EDUCATION==14
6,15,Maestría estudiando,1,EDUCATION==15
7,16,Maestría incompleta,1,EDUCATION==16
8,17,Doctorado completo,0,EDUCATION==17
9,18,Doctorado estudiando,0,EDUCATION==18


In [49]:
facets_dfs['SECTOR']

,option_id,option_name,number_of_matches,filter_data
0,2,Agricultura/Ganadería,37,SECTOR==2
1,3,Agropecuaria,14,SECTOR==3
2,4,Arquitectura,4,SECTOR==4
3,5,Automotriz,16,SECTOR==5
4,6,Autopistas,0,SECTOR==6
...,...,...,...,...
61,43,"Suministro de electricidad, gas y agua",14,SECTOR==43
62,63,Tecnologías de la información y Sistemas,17,SECTOR==63
63,64,Telecomunicaciones,77,SECTOR==64
64,65,Textil,18,SECTOR==65


In [93]:
facets_dfs['HOURS']

,option_id,option_name,number_of_matches,filter_data
0,1,Tiempo completo,1239,HOURS==1
1,2,Tiempo parcial,64,HOURS==2
2,3,Por horas,10,HOURS==3
